In [1]:
"""
Statistics: count difference between anti-deuteron and anti-proton
@aurthor Jiancheng Zeng
@July 15 2021

"""

'\nStatistics: count difference between anti-deuteron and anti-proton\n@aurthor Jiancheng Zeng\n@July 15 2021\n\n'

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import statistics
from numpy import array
import pandas as pd

event = []
identical = []
En = 40

"""
Select by name of the column
---------------------
"""
title = ["eventID", "trackID", "parentID", "particleID", "copyNb1 (mother volume)", "copyNb", "material", "time (ns)", "energy(MeV)", "energy deposit(MeV)", "parent process name", "process name", "px (MeV/c)", "py (MeV/c)", "pz (MeV/c)", "step length (cm)", "x (cm)", "y (cm)", "z (cm)"]

"""
Read data from total data file
--------------------
"""
print("Reading data ...")
#data = pd.read_table("/Users/jiancheng/GRAMS/Toy Model/Machine Learning/210824_anti_proton_250MeV_Lin.dat")    #get data file
data = pd.read_table("/Users/jiancheng/CloudDocuments/Documents/Github/Geant4-simulation/Original_2021/output/test_proton.dat")
print("finish reading data!")

"""
modification
"""
print("Modifing data ...")
for i in range(0, len(data.iloc[:,0])):
    if(0 <= data.iloc[i,2] <= 1):
        event.append(list(data.iloc[i,:]))
    print("Modifing process:"+str(int(100*i/len(data.iloc[:,0])))+"%", end='\r')
event = np.array(event)


"""
recognize the data file length distribution
"""
print("\n Recognize events ...")
s = 0
for s in range(len(event[:,0])-1):
    if(int(event[s,0]) != int(event[s+1,0])):
        identical.append(s+1)
    print("Recognizing process:"+str(int(100*s/len(event[:,0])))+"%", end='\r')
    #time.sleep(0.001)
selection = np.split(event, identical)

identical.append(len(event))

np.savetxt('/Users/jiancheng/GRAMS/Toy Model/Machine Learning/test/test.txt', selection, fmt='%s')

print("\n finished!")

Reading data ...
finish reading data!
Modifing data ...
Modifing process:99%
 Recognize events ...
Recognizing process:99%
 finished!


In [12]:
import math
PeD = [None] * len(identical)    #Penetration depth
EnD = [None] * len(identical)    #Energy deposit
NuPip = [0] * len(identical)     #Number of pion+
NuPim = [0] * len(identical)     #Number of pion-
NuPi0 = [0] * len(identical)     #Number of neutral pion
NuPiC = [0] * len(identical)     #Number of charged pion
NuP = [0] * len(identical)       #Number of proton
XRay = [0] * len(identical)      #X-ray
lDetector = 2000                 #detector depth
temp = 0
step = [0]
inflight = []
step.extend(identical)
print(len(identical))
"""
find penatration depth, energy deposite and number of pions
"""
for i in range(len(identical)):
    for j in range(step[i+1] - step[i]):
        if(event[step[i] + j][3] != '-2212'):
            PeD[i] = math.sqrt((lDetector - float(event[temp + j - 1][18])) ** 2 + (float(event[temp + j - 1][17]) ** 2) + (float(event[temp + j - 1][16]) ** 2))
            EnD[i] = float(event[step[i]][8])
            if(event[temp + j - 1][8] != event[temp + j - 1][9]): inflight.append(i)
            break
    for k in range(step[i+1] - step[i] - 1):
        if(event[temp + k][1] != event[temp + k + 1][1]):
            if(event[temp + k][3] == '211'):
                NuPip[i] = NuPip[i] + 1
                NuPiC[i] = NuPiC[i] + 1
            elif(event[temp + k][3] == '-211'):
                NuPim[i] = NuPim[i] + 1
                NuPiC[i] = NuPiC[i] + 1
            elif(event[temp + k][3] == '111'):
                NuPi0[i] = NuPi0[i] + 1
            elif(event[temp + k][3] == '2212'):
                NuP[i] = NuP[i] + 1
    if(event[identical[i] - 1][3] == '211'):    #specify last track
        NuPip[i] = NuPip[i] + 1
        NuPiC[i] = NuPiC[i] + 1
    elif(event[identical[i] - 1][3] == '-211'):
        NuPim[i] = NuPim[i] + 1
        NuPiC[i] = NuPiC[i] + 1
    elif(event[identical[i] - 1][3] == '111'):
        NuPi0[i] = NuPi0[i] + 1
    elif(event[identical[i] - 1][3] == '2212'):
        NuP[i] = NuP[i] + 1
    temp = identical[i]
    print("forming vectors...:"+str(int(100*i/len(identical)))+"%", end='\r')
    

"""
print("penetration depth:")
print(PeD)
print("\n number of pi+:")
print(NuPip)
print("\n number of pi-:")
print(NuPim)
print("\n number of pi0:")
print(NuPi0)
print("\n number of proton:")
print(NuP)
"""

vector=[PeD, EnD, NuPiC, NuP]    #vector content follow by: Penetration depth, energy diposit, number of charged pions, number of protons
vector=np.transpose(vector)
vector = pd.DataFrame(vector)
vector = vector.drop(labels = inflight)

np.savetxt('/Users/jiancheng/GRAMS/Toy Model/Machine Learning/test/test_p_vector.txt', vector, delimiter = '\t ', fmt='%.6s')
print("\n Done! Vector saved!")


500
forming vectors...:99%
 Done! Vector saved!
